In [ ]:
%cd ..

In [ ]:
from os import getenv

from uuid import uuid4

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.messages import HumanMessage, AIMessage, SystemMessage

from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(
  api_key=getenv("OPENAI_API_KEY"),
  base_url="https://openrouter.ai/api/v1",
  model="gpt-4o-mini",
)

llm.invoke(input=[SystemMessage(content="You are a helpful assistant"), HumanMessage(content="Hi, help me with coding.")], config={"configurable": {"thread_id": str(uuid4()) }})


In [ ]:
from uuid import uuid4
import random

from typing import Literal, Optional, Annotated
from typing_extensions import TypedDict

from langchain.tools import tool

from langgraph.checkpoint.base import BaseCheckpointSaver
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.language_models import BaseChatModel
from langchain_core.messages import AnyMessage, ToolMessage, SystemMessage, AIMessage, HumanMessage

from langgraph.types import interrupt, Command
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [ ]:
@tool("get_weather", description="Get the current weather in a given location as string.")
def get_weather(location):
    weather = random.choice(["sunny", "rainy"])
    return f"The weather in {location} is {weather}."

@tool("ask_for_help", description="Ask for help from the caller agent (usually human).")
def ask_for_help(question):
    help = interrupt(f"I have a question : {question}")
    return help 

@tool("ask_for_help_tkt", description="Ask for help from the caller agent (usually travel agent).")
def ask_for_help_tkt(question):
    help = interrupt(f"I have a question : {question}")
    return help 

@tool("calculate_total", description="Calculate the total amount from quantity and unit price.")
def calculate_total(quantity: int, unit_price: float) -> float:
    return quantity * unit_price

@tool("check_ticket_price", description="Check the price of a ticket for a given origin, destination and whether it is round-trip or one-way.")
def check_ticket_price(origin: str, destination: str, round_trip: bool) -> float:
    base_price = 100.0
    if round_trip:
        return base_price * 2
    return base_price

class TravelAgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    loop_counter: int

class TicketingAgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    loop_counter: int

In [ ]:
def create_ticketing_agent(llm: BaseChatModel, checkpointer: BaseCheckpointSaver):
    MAX_LOOPS = 3
    tools = [ask_for_help_tkt, calculate_total, check_ticket_price]
    tool_name_to_executables = {tool.name: tool for tool in tools}
    llm_with_tools = llm.bind_tools(tools)

    def llm_call(state: TicketingAgentState) -> TicketingAgentState:
        if state.get("loop_counter", 0) > MAX_LOOPS:
            state["messages"] = [AIMessage(content="It seems I am going around. Goodbye!")]
            return state

        _input = [SystemMessage("You are a ticketing agent. \
                                You can help calculate the total ticket price. And print invoice."
                            )] + \
                 state["messages"]

        ai_message = llm_with_tools.invoke(
                input=_input
            )
        state["messages"] = [ai_message]
        state["loop_counter"] = 1 + state.get("loop_counter", 0)
        return state

    def check_tool_call(state: TicketingAgentState) -> Literal["tool_executor", "END"]:
        last_message = state["messages"][-1]
        if last_message.tool_calls:
            return "tool_executor"
        else:
            return END

    def tool_executor(state: TicketingAgentState):
        result = []
        for tc in state["messages"][-1].tool_calls:
            _id, _name, _args = tc["id"], tc["name"], tc["args"]
            tool_response = tool_name_to_executables[_name].invoke(_args)
            result.append(ToolMessage(content=tool_response, tool_call_id=_id))
        state["messages"].extend(result)
        return state

    builder = StateGraph(TicketingAgentState)
    builder.add_node("llm_call", llm_call)
    builder.add_node("tool_executor", tool_executor)
    builder.add_edge(START, "llm_call")
    builder.add_conditional_edges("llm_call", check_tool_call, ["tool_executor", END])
    builder.add_edge("tool_executor", "llm_call")

    return builder.compile(checkpointer=checkpointer)


In [ ]:

ticketing_llm = ChatOpenAI(
  api_key=getenv("OPENAI_API_KEY"),
  base_url="https://openrouter.ai/api/v1",
  model="gpt-4o-mini",
)

user_inputs = [
    "I want to go to Bali, Indonesia. 2 tickets.",
    "Sydney, Australia. Round-trip please. ",
    "yes. Proceed and invpoice me",
    "...",
    "exit",
    "exit"
]
input_iter = iter(user_inputs)

greeting = "Hi! I'm your ticketing assistant. How can I help you today?\n"
print (greeting)
config={"configurable": {"thread_id": str(uuid4()) }}
agent_tkt = create_ticketing_agent(ticketing_llm, InMemorySaver())
while True:    
    content = next(input_iter) # input("Your response (or 'exit' to quit): ")
    if content.lower() in ['exit', 'quit']:
        print("Goodbye!")
        break
    human_message = HumanMessage(content=content)
    human_message.pretty_print()
    response = agent_tkt.invoke({"messages": [human_message], "loop_counter": 0}, config=config)
    while "__interrupt__" in response:
        response["messages"][-1].pretty_print()
        human_response = next(input_iter) # input (str(response["messages"][-1].content) + "\nYour response: ")
        response = agent_tkt.invoke(Command(resume=response), config=config)

    response["messages"][-1].pretty_print()

In [ ]:
from typing import Annotated
from langchain.tools import InjectedToolCallId, ToolRuntime
from langgraph.types import Command


@tool(
    "ticketing_agent_tool",
    description="A tool for interacting with the ticketing agent. \
        Use this tool to delegate ticketing-related queries to the ticketing agent. \
        Make sure you provide what you have to the agent (e.g., number of tickets, \
        round trip or one way, origin and destination) so it has the full context.",
)
# We need to pass the `tool_call_id` to the sub agent so it can use it to respond with the tool call result
def agent_tkt_tool(
    query: str, 
    # tool_call_id: Annotated[str, InjectedToolCallId],
# You need to return a `Command` object to include more than just a final tool call
) -> str:
    result = agent_tkt.invoke({
        "messages": [{"role": "user", "content": query}]
    })
    while "__interrupt__" in result:
        human_response = interrupt("Ticketing Agent has a question: \n" + str(result["messages"][-1].content) + "\nYour response: ")
        _config = {"configurable": {"thread_id": str(uuid4()) }}
        result = agent_tkt.invoke(Command(resume=human_response), config=_config)
    return result["messages"][-1].content
    # return Command(update={
    #     # This is the example state key we are passing back
    #     # "example_state_key": result["example_state_key"],
    #     "messages": [
    #         ToolMessage(
    #             content=result["messages"][-1].content,
    #             # We need to include the tool call id so it matches up with the right tool call
    #             tool_call_id=tool_call_id
    #         )
    #     ]
    # })

In [ ]:
from langchain_core.language_models import BaseChatModel
from langgraph.checkpoint.base import BaseCheckpointSaver

def create_travel_agent(llm: BaseChatModel, checkpointer: BaseCheckpointSaver):
    tools = [get_weather, ask_for_help, agent_tkt_tool]
    tool_name_to_executables = {tool.name: tool for tool in tools}
    llm_with_tools = llm.bind_tools(tools)
    MAX_LOOPS = 3
    def llm_call(state: TravelAgentState) -> TravelAgentState:
        if state.get("loop_counter", 0) > MAX_LOOPS:
            state["messages"] = [AIMessage(content="It seems I am going around. Goodbye!")]
            return state

        _input = [SystemMessage("You are a travel agent. You can help the caller check weather information. You can also recommend hotels.")] + \
                 state["messages"]

        ai_message = llm_with_tools.invoke(
                input=_input
            )
        state["messages"] = [ai_message]
        state["loop_counter"] = 1 + state.get("loop_counter", 0)
        return state

    def check_tool_call(state: TravelAgentState) -> Literal["tool_executor", "END"]:
        last_message = state["messages"][-1]
        if last_message.tool_calls:
            return "tool_executor"
        else:
            return END

    def tool_executor(state: TravelAgentState):
        result = []
        for tc in state["messages"][-1].tool_calls:
            _id, _name, _args = tc["id"], tc["name"], tc["args"]
            tool_response = tool_name_to_executables[_name].invoke(_args)
            result.append(ToolMessage(content=tool_response, tool_call_id=_id))
        state["messages"].extend(result)
        return state

    builder = StateGraph(TravelAgentState)
    builder.add_node("llm_call", llm_call)
    builder.add_node("tool_executor", tool_executor)
    builder.add_edge(START, "llm_call")
    builder.add_conditional_edges("llm_call", check_tool_call, ["tool_executor", END])
    builder.add_edge("tool_executor", "llm_call")

    return builder.compile(checkpointer=checkpointer)

In [ ]:
user_inputs = [
    "I want to go to Bali. For 2 adults. Next week Wednesday.",
    "Departing from Sydney Australia, 30th November return. Nothing else. ",
    "yes, proceed with the booking",
    "ok,",
    "exit",
    "exit"
]
input_iter = iter(user_inputs)

llm_travel = ChatOpenAI(
  api_key=getenv("OPENAI_API_KEY"),
  base_url="https://openrouter.ai/api/v1",
  model="gpt-4o-mini",
)

greeting = "Hi! I'm your travel assistant. How can I help you today?\n"
print (greeting)
config={"configurable": {"thread_id": str(uuid4()) }}
agent = create_travel_agent(llm_travel, InMemorySaver())
while True:    
    content = next(input_iter) # input("Your response (or 'exit' to quit):") # next(input_iter) # input("Your response (or 'exit' to quit): ")
    if content.lower() in ['exit', 'quit']:
        print("Goodbye!")
        break
    human_message = HumanMessage(content=content)
    human_message.pretty_print()
    response = agent.invoke({"messages": [human_message], "loop_counter": 0}, config=config)
    while "__interrupt__" in response:
        response["messages"][-1].pretty_print()
        human_response = next(input_iter) # input (str(response["messages"][-1].content) + "\nYour response: ")
        response = agent.invoke(Command(resume=response), config=config)

    response["messages"][-1].pretty_print()